# SciKit-RT workshop 19/01/21

## Images

In [ ]:
# Set path to sample data
data_path = "/Users/hannahpullen/Work/Workshop/sample_data"

### Loading an image from a dicom file

In [ ]:
from skrt import Image

im = Image(f"{data_path}/head_and_neck/CT/image")
im

### Press `tab` to see possible functions

In [ ]:
im.

In [ ]:
# Examine its properties
print("Voxel size:", im.get_voxel_size())
print("Origin:", im.get_origin())
print("Number of voxels:", im.get_n_voxels())

In [ ]:
# Access the image array
data = im.get_data()
data.shape

### Plotting an image

In [ ]:
im.plot()

### Customising the plot

In [ ]:
# Use python's "help" function for documentation
help(im.plot)

In [ ]:
# Plotting with extra arguments
im.plot(
    pos=20,
    annotate_slice='lime',
    zoom=1.5,
    colorbar=True,
    title='Head and neck CT scan',
    major_ticks=100,
    minor_ticks=10
)

### Interactive plotting with `view()`

In [ ]:
im.view(colorbar=True)

### Writing an image

In [ ]:
im.write("my_image.nii")    # nifti
im.write("my_image_dicom")  # dicom

### Comparing two images

In [ ]:
# Load MVCT scans from different treatment days
im1 = Image(f"{data_path}/head_and_neck/MVCT/1/image", title="MVCT 1")
im2 = Image(f"{data_path}/head_and_neck/MVCT/2/image", title="MVCT 2")

In [ ]:
# View side-by-side
im1.view(images=im2)

In [ ]:
# Create a comparison image
im1.view(images=im2, comparison=True)

In [ ]:
# View the comparison image only
im1.view(images=im2, comparison=True, comparison_only=True)

### Image manipulation

In [ ]:
# Transforming
im2 = im.clone()  # Make a clone of original image so we don't change it
im2.transform(translation=[100, 100, 0], rotation=[0, 0, 45])
im2.plot()

In [ ]:
# Resampling
im2 = im.clone()
im2.resample(voxel_size=[10, 10, None])
im2.plot()

In [ ]:
# Cropping
im2 = im.clone()
im2.crop(xlim=[-80, 100], ylim=[-150, 100])
im2.plot()

## Structure sets and ROIs

### Load a structure set from dicom

In [ ]:
from skrt import StructureSet

structs = StructureSet(f"{data_path}/head_and_neck/CT/RTSTRUCT.dcm")
structs

### Print ROI names

In [ ]:
structs.print_rois()

### View the entire structure set

In [ ]:
structs.view()

### Access a single ROI

In [ ]:
rois = structs.get_rois()
rois

In [ ]:
# Access by name
roi = structs.get_roi("right parotid")
roi.view(roi_plot_type='filled')

### Access the raw data of an ROI
Two ways to represent an ROI:
- Lists of contour points on each slice
- Binary mask

In [ ]:
# Contours
contours = roi.get_contours()
contours

In [ ]:
# Can also get contours in different orientations: 'x-y' (axial), 'x-z' (coronal), or 'y-z' (sagittal)
contours_xz = roi.get_contours('x-z')
roi.plot('x-z')

In [ ]:
# Binary mask
mask = roi.get_mask()
mask

In [ ]:
roi.plot(plot_type="mask")

In [ ]:
# Plot mask and contour together
roi.plot(plot_type="filled")

In [ ]:
# Interactive viewing
roi.view()

### Get the geometric properties of an ROI

In [ ]:
roi.get_

In [ ]:
print("Volume:", roi.get_volume())
print("Area on central slice:", roi.get_area())
print("Centroid position:", roi.get_centroid())

In [ ]:
# Get multiple properties in a table
roi.get_geometry()

In [ ]:
help(roi.get_geometry)

### Get geometric properties of all ROIs in the structure set

In [ ]:
structs.get_geometry()

In [ ]:
structs.get_geometry(
    metrics=["volume", "area", "length_z"], 
    decimal_places=2, 
    vol_units="ml"
)

In [ ]:
# This output is a pandas DataFrame - can write to csv, tex, etc
df = structs.get_geometry()
df.to_csv('geometric_data.csv')

### Assigning an Image to a StructureSet

Two ways:
- Use the `image` argument when creating the StructureSet
- Use the `set_image()` function on the StructureSet

In [ ]:
from skrt import StructureSet
structs = StructureSet(f"{data_path}/head_and_neck/CT/RTSTRUCT.dcm", image=im)

In [ ]:
# View interactively with image
structs.view()

In [ ]:
# View interactively with geometric data
structs.view(roi_info=True)

### Comparing ROIs

In [ ]:
# Extract two ROIs from the structure set and compare them
right_parotid = structs.get_roi("right parotid")
left_parotid = structs.get_roi("left parotid")

In [ ]:
right_parotid.get_relative_area_diff(left_parotid)

In [ ]:
print("Distance:", right_parotid.get_abs_centroid_distance(left_parotid))
print("Dice score:", right_parotid.get_dice(left_parotid))

In [ ]:
# Can also get comparison table as a DataFrame
right_parotid.get_comparison(left_parotid)

### Comparing one structure set to another

In [ ]:
# Let's load one of the MVCT structure sets and print its ROIs...
structs_mv = StructureSet(f"{data_path}/head_and_neck/MVCT/1/RTSTRUCT.dcm")
structs_mv.print_rois()

In [ ]:
# Let's look again at the ROIs in the kVCT structure set...
structs.print_rois()

### Renaming and filtering ROIs

In [ ]:
# Make a naming dictionary (note: case insensitive, * for wildcard)
# Maps desired name : potential input names

names = {
    'MPC': 'mpc',
    'Oral cavity': 'oral*cavity',
    'SPC': 'spc',
    'SG larynx': 'sg*',
    'Left parotid': ['left*parotid', 'parotid*left'],
    'Right parotid': ['right*parotid', 'parotid*right']
}

In [ ]:
# Make a filtered copy of original structure set
structs_kv = structs.filtered_copy(names=names, keep_renamed_only=True)
structs_kv.print_rois()
structs_kv.view()

In [ ]:
# Could also load the StructureSet with the naming dictionary
structs_mv = StructureSet(
    f"{data_path}/head_and_neck/MVCT/1/RTSTRUCT.dcm",  
    names=names, 
    keep_renamed_only=True,
    ignore_dicom_colors=True,
    image=im
)
structs_mv.print_rois()
structs_mv.view()

In [ ]:
# Get comparison table
structs_kv.get_comparison(
    structs_mv, 
    metrics=["dice", "centroid"],
    decimal_places=2
)

In [ ]:
# Plot comparisons
roi = structs_kv.get_roi("Oral cavity")
roi2 = structs_mv.get_roi("Oral cavity")
roi.plot_comparison(roi2, names=['kVCT', 'MVCT'], plot_type='centroid')

In [ ]:
# Interactively view both structure sets
im.view(rois=[structs_kv, structs_mv], compare_rois=['dice', 'dice_slice'])

### Consensus contours

In [ ]:
# Load ROIs from multiple nifti files
from skrt import StructureSet

iov = StructureSet(f"{data_path}/rectum_iov/")
iov.print_rois()

In [ ]:
iov.plot(plot_type="contour", idx=7)

In [ ]:
staple = iov.get_staple()
staple.plot(idx=7)

In [ ]:
# Quick consensus plotting functions
iov.plot_consensus('majority', rois_in_background=True, color='blue', idx=7)

In [ ]:
# Interactively compare each contour to the consensus of all others
iov.view(roi_plot_type='contour', roi_consensus=True, compare_rois=['dice', 'dice_slice'])

In [ ]:
# Compare every ROI to the consensus of all others
iov.get_comparison(comp_type='consensus', consensus_type='majority')

### Loading ROIs from a multi-label array

In [ ]:
# Try loading the output of an InnerEye segmentation...
from skrt import StructureSet
structs_innereye = StructureSet(f"{data_path}/innereye/segmentation.nii.gz", 
                                image="sample_data/innereye/MVCT.nii.gz")
structs_innereye.view(roi_plot_type='mask')

In [ ]:
# Load with the multi_label option
structs_innereye = StructureSet(f"{data_path}/innereye/segmentation.nii.gz", 
                                image=f"{data_path}/innereye/MVCT.nii.gz",
                                multi_label=True)
structs_innereye.view(roi_plot_type='mask')

In [ ]:
# Set custom names
names = ["bladder", "femur1", "femur2", "prostate", "rectum"]
structs_innereye = StructureSet(f"{data_path}/innereye/segmentation.nii.gz", 
                                image=f"{data_path}/innereye/MVCT.nii.gz",
                                multi_label=True,
                                names=names
                               )
structs_innereye.view(roi_plot_type='mask')

## Dose maps

### Load a dose map from dicom

In [ ]:
from skrt import Dose

dose = Dose(f"{data_path}/head_and_neck/CT/RTDOSE.dcm")
dose.view(colorbar=True)

In [ ]:
# Assign an image and view overlay
dose.set_image(im)
dose.view(include_image=True, title="Image with dose")

### Get dose inside an ROI

In [ ]:
# Get mean dose in left parotid 

roi = structs_kv.get_roi("Left parotid")

print("mean dose:", dose.get_mean_dose(roi))

In [ ]:
# Get dose inside every voxel in the ROI
print(dose.get_dose_in_roi(roi))

### Plot dose-volume histogram

In [ ]:
dose.plot_DVH(roi)

## Image registration

- Making a Registration object
- Viewing and adding default parameter files
- Running the registration, viewing results
- Transforming a dose field (then compare)
- Transforming a structure set one way
- Compare
- Get consensus of both ways

### Create a Registration object

In [ ]:
# First, set elastix directory if needed
from skrt.registration import set_elastix_dir
set_elastix_dir("/Users/hannahpullen/elastix-5.0.1-mac/")

In [ ]:
# Create a Registration object
from skrt.registration import Registration

reg = Registration(
    'mvct_fixed',
    fixed=f'{data_path}/head_and_neck/MVCT/1/image/',
    moving=f'{data_path}/head_and_neck/CT/image/',
    overwrite=True  # Remove any existing data
)

In [ ]:
# View the contents of the registration
reg

In [ ]:
# View input images
reg.view_init()

In [ ]:
# View registration steps (none added yet!)
reg.steps

In [ ]:
# Add a step
elastix_dir = "/Users/hannahpullen/Dropbox/scikit-rt workshop/elastix_params"
reg.clear()
reg.add_pfile(f"{elastix_dir}/MI_Translation.txt", name="translation")

In [ ]:
reg.steps

In [ ]:
# Inspect the contents of the input file
reg.get_input_parameters("translation")

In [ ]:
# Adjust a parameter
reg.adjust_pfile("translation", params={"MaximumNumberOfIterations": 400})
reg.get_input_parameters("translation")

In [ ]:
# Perform registration
reg.register()

In [ ]:
# View the results
reg.view_result()

In [ ]:
# Maybe we need to add another step! Add a BSpline (deformable) step
reg.add_pfile(f"{elastix_dir}/MI_BSpline30.txt", name="bspline")
reg.steps

In [ ]:
reg.capture_output = True
reg.view_result()

In [ ]:
# Load a pre-existing registration result
reg = Registration("mvct_fixed")
reg

### Transforming dose fields

In [ ]:
from skrt import Dose

dose_kv = Dose(f"{data_path}/head_and_neck/CT/RTDOSE.dcm")
reg.capture_output = True
dose_transformed = reg.transform(dose_kv)

dose_mv = Dose(f"{data_path}/head_and_neck/MVCT/1/RTDOSE.dcm")
dose_transformed.view(images=dose_mv, comparison=True)

### Transforming StructureSets
Image registration gives us a transform that maps each position in the **fixed image** to its corresponding location in the **moving image**. This gives us two ways to transform contours:

1. **Pull** contours from moving image to fixed image by resampling the contour binary mask into the fixed image's frame of reference.

2. **Push** contours from fixed image to moving image by applying the transform to the contour points.

In [ ]:
# Resample the kVCT structures (as binary masks) into the MVCT domain
from skrt import StructureSet

names = {
    'MPC': 'mpc',
    'Oral cavity': 'oral*cavity',
    'SPC': 'spc',
    'SG larynx': 'sg*',
    'Left parotid': ['left*parotid', 'parotid*left'],
    'Right parotid': ['right*parotid', 'parotid*right']
}
structs_kv = StructureSet(f"{data_path}/head_and_neck/CT/RTSTRUCT.dcm", names=names, keep_renamed_only=True)
structs_kv_transformed = reg.transform(structs_kv)

In [ ]:
structs_kv_transformed.view()

In [ ]:
# Compare the propagated ROIs with the manually drawn ROIs
structs_mv = StructureSet(f"{data_path}/head_and_neck/MVCT/1/RTSTRUCT.dcm", names=names, image=reg.fixed_image,
                          ignore_dicom_colors=True)
structs_mv.view(rois=structs_kv_transformed, compare_rois=True)

### Transforming contour points

In [ ]:
# Need to find a transform in the opposite direction (kVCT -> MVCT)
reg2 = Registration(
    'kvct_fixed',
    fixed=f'{data_path}/head_and_neck/CT/image/',
    moving=f'{data_path}/head_and_neck/MVCT/1/image/',
    overwrite=True,
    pfiles=reg.pfiles  # Copy parameter files from first registration
)

In [ ]:
reg2.register()

In [ ]:
reg2.view_result()

In [ ]:
# Transform structure points
reg2.capture_output = True
structs_kv2 = reg2.transform(structs_kv, transform_points=True)
structs_mv.view(rois=structs_kv2, compare_rois=True)

In [ ]:
# Or get pandas DataFrame - more useful for non-interactive work
structs_mv.get_comparison(structs_kv2)

In [ ]:
# Compare both sets of transformed ROIs
structs_kv2.view(rois=[structs_kv_transformed], compare_rois=['dice', 'dice_slice'])

In [ ]:
# Make new StructureSet containing consensus of the two transforms
combined = StructureSet()
for name in structs_kv2.get_roi_names():
    pair = StructureSet([structs_kv2.get_roi(name), structs_kv_transformed.get_roi(name)])
    staple = pair.get_overlap()
    staple.name = name
    combined.add_roi(staple)

In [ ]:
combined.set_image(structs_kv2.image)
combined.view(legend=True)